# Openshift with WatsonX and PosgreSQL for RAG

In [3]:
from IPython.display import clear_output
!pip install --upgrade pip
!pip install "langchain==0.0.345" 
!pip install wget 
!pip install sentence-transformers 
!pip install "chromadb==0.3.26" 
!pip install "ibm-watson-machine-learning>=1.0.333" 
!pip install pydantic==1.10.11
!pip install python-dotenv
!pip install typing-inspect==0.8.0
#!pip install typing_extensions==4.5.0
!pip install psycopg2-binary
!pip install pypdf
!pip install pgvector

clear_output()

In [4]:
#!pip install langchain_community
#!pip install langchain ibm-watson-machine-learning --upgrade

In [5]:
import os, getpass
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
project_id = os.getenv("PROJECT_ID", None)
credentials = {
    #"url":  "https://eu-de.ml.cloud.ibm.com",
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": os.getenv("API_KEY", None)
}

In [ ]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

In [ ]:
import wget
filename = 'state_of_the_union.txt'
url = 'https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/foundation_models/state_of_the_union.txt'
if not os.path.isfile(filename):
    wget.download(url, out=filename)

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
loader = TextLoader(filename ,encoding='utf-8')
documents = loader.load()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [ ]:
import os
from dotenv import load_dotenv
# Load the .env file
load_dotenv()
# Get the values from the .env file
user = os.getenv("user")
password = os.getenv("password")
database = os.getenv("database")
server = os.getenv("server")
print("User:", user)
print("Database:", database)

In [ ]:
# Construct the connection string
CONNECTION_STRING = f"postgresql+psycopg://{user}:{password}@{server}/{database}"
#CONNECTION_STRING = f"postgresql://{user}:{password}@{server}/{database}"
# Print the connection string
#print(CONNECTION_STRING)

In [ ]:
user = "testuser"
password ="testpwd"
database = "vectordb"
#server = "af651cca01b154fe28a0df0167cad5a7-844854289.us-east-2.elb.amazonaws.com"
server="localhost"
# Construct the connection string
CONNECTION_STRING = f"postgresql+psycopg://{user}:{password}@{server}:5432/{database}"
# Print the connection string
print(CONNECTION_STRING)

In [ ]:
import psycopg2

conn = psycopg2.connect(
    host=server,
    database=database,
    user=user,
    password=password
)

cur = conn.cursor()
cur.execute("SELECT 1")
print(cur.fetchone())  # Should print (1,)
conn.close()


In [ ]:
import os
from dotenv import load_dotenv
import psycopg2

# Construct the connection string
CONNECTION_STRING = f"postgresql://{user}:{password}@{server}/{database}"
print(CONNECTION_STRING)

In [ ]:
# Create a connection to the database
conn = psycopg2.connect(CONNECTION_STRING)
# Create a cursor object to execute queries
cur = conn.cursor()
# Execute the SQL command
cur.execute("""
    CREATE EXTENSION IF NOT EXISTS vector;
    CREATE TABLE IF NOT EXISTS embeddings (
      id SERIAL PRIMARY KEY,
      embedding vector,
      text text,
      created_at timestamptz DEFAULT now()
    );
""")

# Commit the changes
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

In [ ]:
# Create a connection to the database
conn = psycopg2.connect(CONNECTION_STRING)

# Create a cursor object to execute queries
cur = conn.cursor()

# Check if the table exists
cur.execute("SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = 'embeddings')")
table_exists = cur.fetchone()[0]

if table_exists:
    print("Table 'embeddings' exists!")
else:
    print("Table 'embeddings' does not exist.")

# Get the schema of the table
cur.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'embeddings'")
schema = cur.fetchall()

print("Schema of table 'embeddings':")
for column in schema:
    print(f"  {column[0]}: {column[1]}")

# Close the cursor and connection
cur.close()
conn.close()

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores.pgvector import PGVector

In [ ]:
import os
import wget

pdf_folder_path = './rhods-doc'
filename = 'Vector_database.pdf'
url = 'https://github.com/ruslanmv/WatsonX-with-Langchain-PostgreSQL-with-pgvector/raw/master/rhods-doc/Vector_database.pdf'

# Create the directory if it doesn't exist
if not os.path.exists(pdf_folder_path):
    os.makedirs(pdf_folder_path)

full_path = os.path.join(pdf_folder_path, filename)

if not os.path.isfile(full_path):
    wget.download(url, out=full_path)


In [ ]:

loader = PyPDFDirectoryLoader(pdf_folder_path)
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024,
                                               chunk_overlap=40)
all_splits_pdfs = text_splitter.split_documents(docs)


In [ ]:
all_splits_pdfs[0]

In [ ]:
for doc in all_splits_pdfs:
    doc.page_content = doc.page_content.replace('\x00', '')

In [ ]:
embeddings = HuggingFaceEmbeddings()

COLLECTION_NAME = "documents_test"

db = PGVector.from_documents(
    documents=all_splits_pdfs,
    embedding=embeddings,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,)

In [ ]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

In [ ]:
model_id = ModelTypes.GRANITE_13B_CHAT

In [ ]:
model_id ='ibm/granite-13b-chat-v1'

In [1]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods
import os 


In [2]:
import os, getpass
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
project_id = os.getenv("PROJECT_ID", None)
credentials = {
    #"url":  "https://eu-de.ml.cloud.ibm.com",
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": os.getenv("API_KEY", None)
}

In [4]:
# Create an instance of WatsonxLLM
# WatsonxLLM initialization
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE.value,
    GenParams.MAX_NEW_TOKENS: 1000,
    GenParams.MIN_NEW_TOKENS: 50,
    GenParams.TEMPERATURE: 0.7,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}

In [5]:
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 200,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}

In [10]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods
from langchain.llms import WatsonxLLM

ImportError: cannot import name 'WatsonxLLM' from 'langchain.llms' (c:\Users\rusla\.conda\envs\openshift\lib\site-packages\langchain\llms\__init__.py)

In [12]:
from ibm_watsonx_ai import WatsonxLLM

ImportError: cannot import name 'WatsonxLLM' from 'ibm_watsonx_ai' (c:\Users\rusla\.conda\envs\openshift\lib\site-packages\ibm_watsonx_ai\__init__.py)

In [9]:
!pip install ibm_watsonx_ai

  Using cached ibm_watsonx_ai-0.0.5-py3-none-any.whl.metadata (8.2 kB)
Using cached ibm_watsonx_ai-0.0.5-py3-none-any.whl (1.3 MB)


In [7]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

In [45]:
model_id = ModelTypes.GRANITE_13B_CHAT

In [46]:
model_id.value

'ibm/granite-13b-chat-v1'

In [47]:

#!pip install https://www.piwheels.org/simple/ibm-watsonx-ai/ibm_watsonx_ai-0.1.1-py3-none-any.whl


In [48]:
#!pip install https://www.piwheels.org/simple/ibm-watsonx-ai/ibm_watsonx_ai-1.0.6-py3-none-any.whl

In [17]:
from typing import Any, List, Mapping, Optional, Union, Dict
from langchain.llms.base import LLM

In [19]:
from langchain.llms.utils import enforce_stop_tokens

In [20]:
from ibm_watson_machine_learning.foundation_models import Model


In [21]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams


In [22]:
from pydantic import BaseModel, Extra

In [28]:
!pip install langchain
!pip install ibm-watson-machine-learning

In [32]:
load_dotenv()
project_id = os.getenv("PROJECT_ID", None)
creds = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": os.getenv("API_KEY", None)
}

class WatsonxLLM(LLM, BaseModel):
    credentials: Optional[Dict] = None
    model: Optional[str] = None
    params: Optional[Dict] = None
    project_id : Optional[str]=None

    class Config:
        extra = Extra.forbid

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        _params = self.params or {}
        return {
            **{"model": self.model},
            **{"params": _params},
        }
    
    @property
    def _llm_type(self) -> str:
        return "IBM WATSONX"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        params = self.params or {}
        model = Model(model_id=self.model, params=params, credentials=self.credentials, project_id=self.project_id)
        text = model.generate_text(prompt)
        if stop is not None:
            text = enforce_stop_tokens(text, stop)
        return text

In [33]:
params = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 256
}
model = WatsonxLLM(model="google/flan-t5-xxl", credentials=creds, params=params, project_id=project_id)
text = "Where is Germany?"
model(text)

TypeError: Can't instantiate abstract class WatsonxLLM with abstract method __call__

In [34]:


load_dotenv()

project_id = os.getenv("PROJECT_ID", None)
creds = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": os.getenv("API_KEY", None)
}

class WatsonxLLM(LLM, BaseModel):
    credentials: Optional[Dict] = None
    model: Optional[str] = None
    params: Optional[Dict] = None
    project_id: Optional[str] = None

    class Config:
        extra = Extra.forbid

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        _params = self.params or {}
        return {
            **{"model": self.model},
            **{"params": _params},
        }

    @property
    def _llm_type(self) -> str:
        return "IBM WATSONX"

    def __call__(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        params = self.params or {}
        model = Model(model_id=self.model, params=params, credentials=self.credentials, project_id=self.project_id)
        text = model.generate_text(prompt)
        if stop is not None:
            text = enforce_stop_tokens(text, stop)
        return text

params = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 256
}

model = WatsonxLLM(model="google/flan-t5-xxl", credentials=creds, params=params, project_id=project_id)
text = "Where is Germany?"
result = model(text)  # This should call the __call__ method
print(result)


Error: Incorrect padding

In [49]:
#!pip install ibm_watsonx_ai

In [50]:
watsonx_granite = WatsonxLLM(
    model_id=model_id.value,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params=parameters
)

ValidationError: 1 validation error for WatsonxLLM
__root__
  Incorrect padding (type=value_error.)

In [ ]:
!pip install ibm-generative-ai

In [35]:
import os
from typing import Optional, Dict, Mapping, Any, List
from pydantic import Extra
from dotenv import load_dotenv

# Assuming necessary imports for LLM, BaseModel, Model, and GenParams are present
# Adjust imports as needed

load_dotenv()

project_id = os.getenv("PROJECT_ID", None)
creds = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": os.getenv("API_KEY", None)
}

class WatsonxLLM(LLM, BaseModel):
    credentials: Optional[Dict] = None
    model: Optional[str] = None
    params: Optional[Dict] = None
    project_id: Optional[str] = None

    class Config:
        extra = Extra.forbid

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        _params = self.params or {}
        return {
            **{"model": self.model},
            **{"params": _params},
        }

    @property
    def _llm_type(self) -> str:
        return "IBM WATSONX"

    def __call__(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        params = self.params or {}
        model = Model(model_id=self.model, params=params, credentials=self.credentials, project_id=self.project_id)
        text = model.generate_text(prompt)
        if stop is not None:
            text = enforce_stop_tokens(text, stop)
        return text

params = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 256
}

# Assuming the necessary import for Model is present
# Adjust imports as needed

model = WatsonxLLM(model="google/flan-t5-xxl", credentials=creds, params=params, project_id=project_id)
text = "Where is Germany?"
result = model(text)  # This should call the __call__ method
print(result)


Error: Incorrect padding